In [1]:
import tensorflow as tf
import keras

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
import cv2
import numpy as np

In [4]:
def to_grayscale(data):
    data2 = []
    for im in data:
        data2.append(cv2.cvtColor(im, cv2.COLOR_BGR2GRAY))
    data2 = np.array(data2)
    data2 = data2.reshape((-1, 32, 32, 1))
    print(data2.shape)
    return data2
    

In [5]:
x_train_g = to_grayscale(x_train)
x_test_g = to_grayscale(x_test)

(50000, 32, 32, 1)
(10000, 32, 32, 1)


In [6]:
y_train.shape

(50000, 1)

In [7]:
y_train = y_train.reshape((-1,))
y_test = y_test.reshape((-1,))
y_train.shape

(50000,)

In [8]:
x_train_g = x_train_g/255.0
x_test_g = x_test_g/255.0

In [16]:
model2 = keras.models.Sequential([
    keras.Input(shape=(32, 32, 1 )),
    keras.layers.Conv2D(filters = 6, kernel_size=(5,5), strides=1, activation="tanh"),
    keras.layers.AveragePooling2D(pool_size=(2,2), strides=2),
    keras.layers.Conv2D(filters = 16, kernel_size=(5,5), strides=1, activation="tanh"),
    keras.layers.AveragePooling2D(pool_size=(2,2), strides=2),
    keras.layers.Conv2D(filters = 120, kernel_size=(5,5), strides=1, activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(84, activation="tanh"),
    keras.layers.Dense(10, activation="softmax") 
])

In [82]:
model1= keras.models.Sequential([
    keras.Input(shape=(32,32,3)),
    keras.layers.Conv2D(filters=8, kernel_size=(3,3), strides=1, activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=32, activation="relu"),
    keras.layers.Dense(units=10, activation="softmax")
])

In [17]:
model2.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [18]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_2 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_3 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten_1 (Flatten)         (None, 120)              

In [14]:
x_train_g.shape

(50000, 32, 32, 1)

In [13]:
y_train.shape

(50000,)

In [31]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4, verbose=True, restore_best_weights=True, start_from_epoch= 5)

In [32]:
model2.fit(x_train_g, y_train, epochs=50, validation_split=0.2, verbose=True, callbacks=[earlyStop])

Epoch 1/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7967 - accuracy: 0.7195 - val_loss: 1.9844 - val_accuracy: 0.4385
Epoch 2/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7782 - accuracy: 0.7264 - val_loss: 2.0037 - val_accuracy: 0.4388
Epoch 3/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7588 - accuracy: 0.7355 - val_loss: 2.0350 - val_accuracy: 0.4398
Epoch 4/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7458 - accuracy: 0.7377 - val_loss: 2.0675 - val_accuracy: 0.4390
Epoch 5/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7294 - accuracy: 0.7456 - val_loss: 2.0882 - val_accuracy: 0.4372
Epoch 6/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.7164 - accuracy: 0.7498 - val_loss: 2.1279 - val_accuracy: 0.4361
Epoch 7/50
1250/1250 [==============================] - 9s 7ms/step - loss: 0.6993 - accuracy: 0.7548 - val_loss: 2.1608 - val_accuracy:

In [33]:
y_pred = model2.predict(x_test_g)

313/313 [==============================] - 1s 3ms/step


In [34]:
y_pred[:2]

array([[5.93985338e-03, 1.18953958e-01, 1.02158241e-01, 1.06251046e-01,
        9.92047600e-04, 3.22354957e-02, 5.29099628e-02, 5.79209486e-03,
        1.38734777e-06, 5.74765921e-01],
       [2.48961318e-09, 1.86596707e-01, 1.50839174e-09, 2.07164771e-10,
        5.39064016e-10, 2.19665264e-09, 9.65518811e-12, 3.52195741e-07,
        1.91171700e-03, 8.11491251e-01]], dtype=float32)

In [35]:
y_pred = [np.argmax(elem) for elem in y_pred]
y_pred[:2]

[9, 9]

In [36]:
y_test[:2]

array([3, 8], dtype=uint8)

In [37]:
from sklearn.metrics import confusion_matrix , classification_report
print("Classification Report: \n", classification_report(y_test, y_pred))

Classification Report: 
               precision    recall  f1-score   support

           0       0.48      0.45      0.46      1000
           1       0.51      0.51      0.51      1000
           2       0.30      0.34      0.32      1000
           3       0.25      0.24      0.24      1000
           4       0.32      0.31      0.31      1000
           5       0.34      0.35      0.34      1000
           6       0.45      0.47      0.46      1000
           7       0.52      0.48      0.50      1000
           8       0.52      0.52      0.52      1000
           9       0.45      0.45      0.45      1000

    accuracy                           0.41     10000
   macro avg       0.41      0.41      0.41     10000
weighted avg       0.41      0.41      0.41     10000



In [38]:
print(confusion_matrix(y_test, y_pred))

[[450  47 113  28  86  26  40  24 138  48]
 [ 46 507  31  32  24  23  57  20  87 173]
 [ 82  27 339 115 135 105  84  47  45  21]
 [ 46  32 108 236  91 192 131  64  28  72]
 [ 50  32 178  98 314  94  96  80  30  28]
 [ 41  16 131 151  71 350  60 115  33  32]
 [ 29  47  89  98 101  77 469  26  24  40]
 [ 32  23  69  78 100 101  33 485  18  61]
 [119  85  39  41  47  27  31  27 518  66]
 [ 47 183  41  59  26  40  43  43  68 450]]
